In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.ticker import MaxNLocator
import sys
import torch

# Add module to path
module_path = Path.cwd().parents[1] / "module"  
sys.path.append(str(module_path))

from pool_utils import PINNDataset, test_dataset_from_true_data
from visualization import compute_relative_errors_from_dict

# Path to repo root
repo_root = Path.cwd().parents[1] 



# --------------------
# ---- True data -----
# --------------------

R = 5
L = 24 
test_dataset, x_max, u_a = test_dataset_from_true_data(R, L, device=torch.device("cpu"))
all_inputs_test_scaled = test_dataset.inputs
all_outputs_test_scaled = test_dataset.outputs

# ----------------------
# ---- Predictions -----
# ----------------------

# St Martin La Porte (Vu et al 2013)
# Initial stress
sigma_v = 5e6
K = 0.75
sigma_h = K*sigma_v
l = 1
# Rotation
beta = 45*np.pi/180 
# In situ stress
sigma_v_0 = sigma_v/2*(1+K + (1-K)*np.cos(2*beta))
sigma_h_0 = sigma_v/2*(1+K - (1-K)*np.cos(2*beta))
tau_vh_0 = sigma_v/2*(1-K)*np.sin(2*beta)

# Mechanical parameters to retrieve
Eh = 620e6
Ev = 340e6
Gvh = 200e6
nh = 0.12
nhv = 0.2
nvh = Ev*nhv/Eh

# mode
mode = "extensometer"

# Load results and save 
folder = f'7_sensors_{mode}_mode_Noise_0%'

# Path to data file
load_dir = os.path.join(repo_root, "examples", f"{mode}_mode", "Results", f"{folder}") 
save_dir = os.path.join(repo_root, "examples", f"{mode}_mode", "Results", f"{folder}", "posterior_analysis") 
os.makedirs(save_dir, exist_ok=True)

# Define discrete zones
Zones = [R, 2*R, 3*R, 4*R, 5*R]

# List all seed folders
seed_dirs = [d for d in os.listdir(load_dir) if d.startswith("s_")]
seed_dirs.sort() # not necessary

# Store per-seed errors
Rel_errors_ux_all = []
Rel_errors_uy_all = []

# Layout preferences
plt.rcParams.update({
    "font.size": 7,
    "axes.labelsize": 7,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7
})

colors = ["tab:red", "tab:blue", "tab:green", "tab:purple"]
markers = ["s", "o", "x", "^"]
labels = ["Zone 1", "Zone 2", "Zone 3", "Zone 4"]

# Main loop
for seed_dir in seed_dirs:
    save_path = os.path.join(load_dir, seed_dir, "all_results.pkl")
    if not os.path.exists(save_path):
        print(f"Warning: {save_path} does not exist, skipping")
        continue
    with open(save_path, "rb") as f:
        results_dict = pickle.load(f)

    ux_preds_list = results_dict["all_ux_preds"]
    uy_preds_list = results_dict["all_uy_preds"]
    
    Rel_errors_ux_seed = np.zeros(shape=(len(ux_preds_list), len(Zones)-1))
    Rel_errors_uy_seed = np.zeros(shape=(len(ux_preds_list), len(Zones)-1))

    for step in range(len(ux_preds_list)):

        ux_preds_step = ux_preds_list[step]
        uy_preds_step = uy_preds_list[step]


        for j in range(len(Zones) - 1):
            
            # Define minimum and maximum distances
            tol = 1e-6
            dmin = Zones[j]/x_max + tol
            dmax = Zones[j+1]/x_max - tol

            # -------------------
            # --- TEST DATA -----
            # -------------------

            # Create mask for points between dmin and dmax
            x = all_inputs_test_scaled[:, 0]
            y = all_inputs_test_scaled[:, 1]
            r = np.sqrt(x**2 + y**2)
            mask = (r >= dmin) & (r <= dmax)
            
            # Select points
            selected_inputs_test_scaled = all_inputs_test_scaled[mask]
            selected_outputs_test_scaled = all_outputs_test_scaled[mask]

            ux_preds_step_selected = ux_preds_step[mask].squeeze()
            uy_preds_step_selected = uy_preds_step[mask].squeeze()

            # -------------------
            # --- PREDICTIONS ---
            # -------------------

            ux_rel = torch.norm(ux_preds_step_selected - selected_outputs_test_scaled[:, 0]) / torch.norm(selected_outputs_test_scaled[:, 0]) * 100
            uy_rel = torch.norm(uy_preds_step_selected - selected_outputs_test_scaled[:, 1]) / torch.norm(selected_outputs_test_scaled[:, 1]) * 100

            Rel_errors_ux_seed[step, j] = ux_rel.item()
            Rel_errors_uy_seed[step, j] = uy_rel.item()

    Rel_errors_ux_all.append(Rel_errors_ux_seed[:])
    Rel_errors_uy_all.append(Rel_errors_uy_seed[:])

# ----------------------------
# ----- DATAFRAME ------------
# ----------------------------

# Convert to numpy arrays
Rel_errors_ux_all = np.array(Rel_errors_ux_all)  
Rel_errors_uy_all = np.array(Rel_errors_uy_all)

# Compute mean and std
Rel_errors_ux_mean = Rel_errors_ux_all.mean(axis=0)
Rel_errors_uy_mean = Rel_errors_uy_all.mean(axis=0)
Rel_errors_ux_std = Rel_errors_ux_all.std(axis=0)
Rel_errors_uy_std = Rel_errors_uy_all.std(axis=0)

# ----------------------------
# ----------- CSV ------------
# ----------------------------
rows = []
n_steps, n_zones = Rel_errors_ux_mean.shape
for i in range(1, n_steps):
    for j in range(n_zones):
        rows.append({
            "step": i,
            "Zone": f"Zone {j+1}",
            "ux_mean": Rel_errors_ux_mean[i, j],
            "ux_std": Rel_errors_ux_std[i, j],
            "uy_mean": Rel_errors_uy_mean[i, j],
            "uy_std": Rel_errors_uy_std[i, j],
        })
        
# -----------------------------------------
# ----------- LOG-SCALE PLOTTING ----------
# -----------------------------------------

epsilon = 1e-12  

# --- ux ---
plt.figure(figsize=(8/2.54,6/2.54))
x = np.arange(2, Rel_errors_ux_all.shape[1] + 1)

for j in range(Rel_errors_ux_all.shape[2]):

    # log of all individual values (not mean/std)
    log_vals = np.log10(Rel_errors_ux_all[:, :, j] + epsilon)  

    log_mean = np.mean(log_vals, axis=0)[1:]
    log_std  = np.std(log_vals, axis=0)[1:]

    mean_curve = 10**log_mean
    lower = 10**(log_mean - log_std)
    upper = 10**(log_mean + log_std)

    plt.plot(
        x, mean_curve,
        marker=markers[j], markersize=4,
        color=colors[j], linewidth=1,
        label=labels[j]
    )

    plt.fill_between(x, lower, upper, alpha=0.2, color=colors[j])

plt.yscale("log")
plt.xlabel("Number of sensors")
plt.ylabel("Test relative error (%)")
plt.legend()
plt.grid(True, which="major", linestyle="--", linewidth=0.5)
plt.minorticks_on()
plt.grid(True, which="minor", linestyle=":", linewidth=0.3, alpha=0.5)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.savefig(os.path.join(save_dir, f"test_zone_{mode}_mode_ux.pdf"), bbox_inches="tight", dpi=300)
plt.show()

# --- uy ---
plt.figure(figsize=(8/2.54,6/2.54))
x = np.arange(2, Rel_errors_uy_all.shape[1] + 1)

for j in range(Rel_errors_uy_all.shape[2]):

    log_vals = np.log10(Rel_errors_uy_all[:, :, j] + epsilon)

    log_mean = np.mean(log_vals, axis=0)[1:]
    log_std  = np.std(log_vals, axis=0)[1:]

    mean_curve = 10**log_mean
    lower = 10**(log_mean - log_std)
    upper = 10**(log_mean + log_std)

    plt.plot(
        x, mean_curve,
        marker=markers[j], markersize=4,
        color=colors[j], linewidth=1,
        label=labels[j]
    )

    plt.fill_between(x, lower, upper, alpha=0.2, color=colors[j])

plt.yscale("log")
plt.xlabel("Number of sensors")
plt.ylabel("Test relative error (%)")
# plt.legend()
plt.grid(True, which="major", linestyle="--", linewidth=0.5)
plt.minorticks_on()
plt.grid(True, which="minor", linestyle=":", linewidth=0.3, alpha=0.5)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.savefig(os.path.join(save_dir, f"test_zone_{mode}_mode_uy.pdf"), bbox_inches="tight", dpi=300)
plt.show()